In [21]:
#!pip install pandas
#!pip install seaborn
#!pip install PyCap
from io import StringIO
from urllib.request import urlretrieve
import urllib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import redcap
from redcap import Project
from collections import Counter
import statistics
import numpy as np
from statistics import mode
import requests
%load_ext rpy2

The rpy2 module is not an IPython extension.


## Sets up the main data extraction process

In [89]:
api_url = 'https://redcap.ucsf.edu/api/'
api_key = '80A7F39F726D6CF77B0C240C1C2DFD41'
project = Project(api_url,api_key, verify_ssl=True)
project.export_records(filter_logic='[3_mo_arm_1][eortcquality_of_life_complete]=2')
pd.options.display.max_columns=999
pd.options.display.max_rows=100

In [38]:
print(project.field_names)

['record_id', 'bor', 'bor_sign', 'bor_date', 'bor_pt2', 'hipaa', 'hipaa_name', 'hipaa_sign', 'hipaa_date', 'consent_p1', 'consent_p2', 'consent_p3', 'consent_p4', 'first_name', 'last_name', 'consent_signature', 'signature_date', 'mail', 'idno', 'first_in', 'lastn', 'email', 'dob', 'age_tod', 'cbduse', 'currentuse', 'last_cdb', 'lenght', 'freqcbd', 'reascbd', 'other_use', 'cbdmedical', 'oth_sym', 'desc1', 'overallsym_scorebe', 'overallsym_scoreaf', 'moduse', 'cbd_oth', 'know_strain', 'strain', 'cbd_thc', 'cbd_ratio', 'recommcbd', 'recom_oth', 'obtain', 'obtain_oth', 'cbdchange', 'howchng', 'stop_use', 'stop_useex', 'start_use', 'start_useex', 'cbd_change', 'whatchange', 'whenchange', 'changefrequency', 'percent_crease', 'change_decrease', 'sympt_change', 'anynew', 'newsympt', 'newsympt_oth', 'symp_stop_bin', 'symp_stp', 'cbd_comment', 'eq5d5l_instr1_usa_eng', 'eq5d_mb_5l_usa_eng', 'eq5d5l_copy1_usa_eng', 'eq5d5l_instr2_usa_eng', 'eq5d_sc_5l_usa_eng', 'eq5d5l_copy2_usa_eng', 'eq5d5l_inst

In [171]:
fields_bl = ['record_id','tod','age', 'sex','cbduse','currentuse','last_cdb','lenght','freqcbd','reascbd','other_use','cbdmedical',
             'oth_sym','desc1','overallsym_scorebe','overallsym_scoreaf','moduse','cbd_oth','know_strain','strain','cbd_thc','cbd_ratio','recommcbd',
             'recom_oth','obtain','eq5d_mb_5l_usa_eng','eq5d_sc_5l_usa_eng', 'eq5d_ua_5l_usa_eng','eq5d_pd_5l_usa_eng', 
             'eq5d_ad_5l_usa_eng','eq5d5l_vas2_usa_eng', 'in_pt','dob_qol','tod','c30_1','c30_2','c30_3','c30_4','c30_5','c30_6','c30_7','c30_8','c30_9','c30_10',
             'c30_11','c30_12','c30_13','c30_14','c30_15','c30_16','c30_17','c30_18','c30_19','c30_20','c30_21','c30_22','c30_23','c30_24','c30_25',
             'c30_26','c30_27','c30_28','c30_29','c30_30','bn20_31','bn20_32','bn20_33','bn20_34','bn20_35','bn20_36','bn20_37','bn20_38','bn20_39',
             'bn20_40','bn20_41','bn20_42','bn20_43','bn20_44','bn20_45','bn20_46','bn20_47','bn20_48','bn20_49','bn20_50','ethnicity','education','employment','economic','ethn','race','maritalsts',
             'vitalsts','last_alive','dod','sex','inital_dx','ini_surgery','ini_surgery_type','ini_tum_lateral','ini_tum_location','ini_path_dx','ini_tumor_grade']
#subset = project.export_records(fields=fields_oi)


## Imports the baseline data

Note that this dataframe will contain the diagnosis. 

In [172]:
bl_event=['baseline_arm_1']
baseline_df = project.export_records(format='df', raw_or_label='raw', export_checkbox_labels=True,events=bl_event, fields=fields_bl)
len(baseline_df.index)

107

In [182]:
baseline_df['education'].head()

record_id
201     8.0
202     8.0
203     9.0
204    10.0
205    10.0
Name: education, dtype: float64

In [173]:
baseline_df = baseline_df.reset_index()
baseline_df = baseline_df.set_index('record_id')
baseline_df['visit'] = 0

In [174]:
print(project.field_names)

['record_id', 'bor', 'bor_sign', 'bor_date', 'bor_pt2', 'hipaa', 'hipaa_name', 'hipaa_sign', 'hipaa_date', 'consent_p1', 'consent_p2', 'consent_p3', 'consent_p4', 'first_name', 'last_name', 'consent_signature', 'signature_date', 'mail', 'idno', 'first_in', 'lastn', 'email', 'dob', 'age_tod', 'cbduse', 'currentuse', 'last_cdb', 'lenght', 'freqcbd', 'reascbd', 'other_use', 'cbdmedical', 'oth_sym', 'desc1', 'overallsym_scorebe', 'overallsym_scoreaf', 'moduse', 'cbd_oth', 'know_strain', 'strain', 'cbd_thc', 'cbd_ratio', 'recommcbd', 'recom_oth', 'obtain', 'obtain_oth', 'cbdchange', 'howchng', 'stop_use', 'stop_useex', 'start_use', 'start_useex', 'cbd_change', 'whatchange', 'whenchange', 'changefrequency', 'percent_crease', 'change_decrease', 'sympt_change', 'anynew', 'newsympt', 'newsympt_oth', 'symp_stop_bin', 'symp_stp', 'cbd_comment', 'eq5d5l_instr1_usa_eng', 'eq5d_mb_5l_usa_eng', 'eq5d5l_copy1_usa_eng', 'eq5d5l_instr2_usa_eng', 'eq5d_sc_5l_usa_eng', 'eq5d5l_copy2_usa_eng', 'eq5d5l_inst

## Imports the data for the 3 month follow up

This is because we want to include data of only those who completed the 3 month survey to have a longitudinal study 

In [175]:
fields_fu1 = ['record_id','tod','age', 'sex','cbduse','currentuse','last_cdb','lenght','freqcbd','reascbd','other_use','cbdmedical',
             'oth_sym','desc1','overallsym_scorebe','overallsym_scoreaf','moduse','cbd_oth','know_strain','strain','cbd_thc','cbd_ratio','recommcbd',
             'recom_oth','obtain','obtain_oth','cbdchange','howchng','stop_use','stop_useex','start_use','start_useex','cbd_change','whatchange','whenchange','changefrequency','percent_crease',
              'change_decrease','sympt_change','anynew','newsympt','newsympt_oth','symp_stop_bin','symp_stp','cbd_comment','eq5d_mb_5l_usa_eng','eq5d_sc_5l_usa_eng', 'eq5d_ua_5l_usa_eng',
              'eq5d_pd_5l_usa_eng','eq5d_ad_5l_usa_eng','eq5d5l_vas2_usa_eng', 'in_pt','dob_qol','tod','c30_1','c30_2','c30_3','c30_4','c30_5','c30_6','c30_7','c30_8','c30_9','c30_10',
             'c30_11','c30_12','c30_13','c30_14','c30_15','c30_16','c30_17','c30_18','c30_19','c30_20','c30_21','c30_22','c30_23','c30_24','c30_25',
             'c30_26','c30_27','c30_28','c30_29','c30_30','bn20_31','bn20_32','bn20_33','bn20_34','bn20_35','bn20_36','bn20_37','bn20_38','bn20_39',
             'bn20_40','bn20_41','bn20_42','bn20_43','bn20_44','bn20_45','bn20_46','bn20_47','bn20_48','bn20_49','bn20_50']
fu1_event=['3_mo_arm_1']
fu1_df = project.export_records(format='df', raw_or_label='raw', export_checkbox_labels=True,events=fu1_event, fields=fields_fu1, filter_logic="[eortcquality_of_life_complete]=2")
len(fu1_df)
fu1_df = fu1_df.reset_index()
fu1_df= fu1_df.set_index('record_id')


Checks how many records we have that have completed the 3 month mark

In [176]:
len(fu1_df.index)
fu1_df.head(len(fu1_df.index)) 
fu1_df['visit'] = 1

Starts filtering the baseline data by people who have completed the 3mo mark so we can merge the dataframes later

In [177]:
ids = list(fu1_df.index)
boolean_series = baseline_df.index.isin(ids)
filtered_df = baseline_df[boolean_series]

#Checking that we have the same number of IDs in both dataframes
print(len(filtered_df.index))
filtered_df.head()

64


,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,cbduse,currentuse,last_cdb,lenght,freqcbd,reascbd,other_use,cbdmedical___1,cbdmedical___2,cbdmedical___3,cbdmedical___4,cbdmedical___5,cbdmedical___6,cbdmedical___7,cbdmedical___8,cbdmedical___9,cbdmedical___10,cbdmedical___11,cbdmedical___12,cbdmedical___13,cbdmedical___14,cbdmedical___99,oth_sym,overallsym_scorebe,overallsym_scoreaf,moduse,cbd_oth,know_strain,strain,cbd_thc,cbd_ratio,recommcbd,recom_oth,obtain,eq5d_mb_5l_usa_eng,eq5d_sc_5l_usa_eng,eq5d_ua_5l_usa_eng,eq5d_pd_5l_usa_eng,eq5d_ad_5l_usa_eng,eq5d5l_vas2_usa_eng,in_pt,dob_qol,tod,age,c30_1,c30_2,c30_3,c30_4,c30_5,c30_6,c30_7,c30_8,c30_9,c30_10,c30_11,c30_12,c30_13,c30_14,c30_15,c30_16,c30_17,c30_18,c30_19,c30_20,c30_21,c30_22,c30_23,c30_24,c30_25,c30_26,c30_27,c30_28,c30_29,c30_30,bn20_31,bn20_32,bn20_33,bn20_34,bn20_35,bn20_36,bn20_37,bn20_38,bn20_39,bn20_40,bn20_41,bn20_42,bn20_43,bn20_44,bn20_45,bn20_46,bn20_47,bn20_48,bn20_49,bn20_50,ethnicity___1,ethnicity___2,ethnicity___3,ethnicity___4,ethnicity___5,ethnicity___6,ethnicity___7,education,employment,economic,ethn,race,maritalsts,vitalsts,last_alive,dod,sex,inital_dx,ini_surgery,ini_surgery_type,ini_tum_lateral,ini_tum_location,ini_path_dx,ini_tumor_grade,visit
record_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201,baseline_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,1.0,2.0,95.0,AJ,1953-03-11,2018-08-22,66.062844,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,2.0,1.0,3.0,1.0,3.0,2.0,7.0,6.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,3.0,1.0,2.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,0,0,0,0,1,0,0,8.0,8.0,2.0,1.0,1.0,0.0,0.0,NaN,NaN,0.0,08/09/1995,1.0,1.0,2.0,2.0,Oligodendroglioma,1.0,0
202,baseline_arm_1,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,90.0,SLA,1957-09-06,2018-08-22,61.572791,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,6.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,0,0,0,0,0,0,8.0,2.0,8.0,1.0,4.0,0.0,0.0,NaN,NaN,0.0,01/19/2016,1.0,2.0,2.0,2.0,Oligodendroglioma,1.0,0
203,baseline_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,70.0,RC,1975-11-01,2018-08-22,43.436781,2.0,3.0,2.0,2.0,1.0,2.0,2.0,1.0,NaN,4.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,4.0,1.0,2.0,2.0,1.0,3.0,1.0,NaN,4.0,2.0,4.0,5.0,6.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1,0,0,0,0,0,0,9.0,9.0,4.0,1.0,4.0,0.0,0.0,NaN,NaN,0.0,05/14/2014,1.0,2.0,1.0,4.0,Oligoastrocytoma,1.0,0
204,baseline_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,95.0,EHP,1950-03-30,2018-08-22,69.027997,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,7.0,7.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,0,0,0,0,0,0,10.0,8.0,NaN,1.0,4.0,0.0,0.0,NaN,NaN,1.0,9/25/2005,1.0,1.0,2.0,4.0,Anaplastic Oligodendroglioma,3.0,0
205,baseline_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,3.0,1.0,66.0,jff,1955-02-13,2018-08-29,64.151785,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1,0,0,0,0,0,0,10.0,1.0,5.0,1.0,4.0,0.0,0.0,NaN,NaN,0.0,2/19/1997,1.0,1.0,1.0,2.0,Low grade central nervous system neoplasm,1.0,0


## Creating a Wide Dataframe

In [178]:
wide_df = filtered_df.merge(fu1_df, left_index=True, right_index=True, suffixes=('_bl','_3m'))
wide_df.head()

,redcap_event_name_bl,redcap_repeat_instrument_bl,redcap_repeat_instance_bl,cbduse_bl,currentuse_bl,last_cdb_bl,lenght_bl,freqcbd_bl,reascbd_bl,other_use_bl,cbdmedical___1_bl,cbdmedical___2_bl,cbdmedical___3_bl,cbdmedical___4_bl,cbdmedical___5_bl,cbdmedical___6_bl,cbdmedical___7_bl,cbdmedical___8_bl,cbdmedical___9_bl,cbdmedical___10_bl,cbdmedical___11_bl,cbdmedical___12_bl,cbdmedical___13_bl,cbdmedical___14_bl,cbdmedical___99_bl,oth_sym_bl,overallsym_scorebe_bl,overallsym_scoreaf_bl,moduse_bl,cbd_oth_bl,know_strain_bl,strain_bl,cbd_thc_bl,cbd_ratio_bl,recommcbd_bl,recom_oth_bl,obtain_bl,eq5d_mb_5l_usa_eng_bl,eq5d_sc_5l_usa_eng_bl,eq5d_ua_5l_usa_eng_bl,eq5d_pd_5l_usa_eng_bl,eq5d_ad_5l_usa_eng_bl,eq5d5l_vas2_usa_eng_bl,in_pt_bl,dob_qol_bl,tod_bl,age_bl,c30_1_bl,c30_2_bl,c30_3_bl,c30_4_bl,c30_5_bl,c30_6_bl,c30_7_bl,c30_8_bl,c30_9_bl,c30_10_bl,c30_11_bl,c30_12_bl,c30_13_bl,c30_14_bl,c30_15_bl,c30_16_bl,c30_17_bl,c30_18_bl,c30_19_bl,c30_20_bl,c30_21_bl,c30_22_bl,c30_23_bl,c30_24_bl,c30_25_bl,c30_26_bl,c30_27_bl,c30_28_bl,c30_29_bl,c30_30_bl,bn20_31_bl,bn20_32_bl,bn20_33_bl,bn20_34_bl,bn20_35_bl,bn20_36_bl,bn20_37_bl,bn20_38_bl,bn20_39_bl,bn20_40_bl,bn20_41_bl,bn20_42_bl,bn20_43_bl,bn20_44_bl,bn20_45_bl,bn20_46_bl,bn20_47_bl,bn20_48_bl,bn20_49_bl,bn20_50_bl,ethnicity___1,ethnicity___2,ethnicity___3,ethnicity___4,ethnicity___5,ethnicity___6,ethnicity___7,education,employment,economic,ethn,race,maritalsts,vitalsts,last_alive,dod,sex_bl,inital_dx,ini_surgery,ini_surgery_type,ini_tum_lateral,ini_tum_location,ini_path_dx,ini_tumor_grade,visit_bl,redcap_event_name_3m,redcap_repeat_instrument_3m,redcap_repeat_instance_3m,cbduse_3m,currentuse_3m,last_cdb_3m,lenght_3m,freqcbd_3m,reascbd_3m,other_use_3m,cbdmedical___1_3m,cbdmedical___2_3m,cbdmedical___3_3m,cbdmedical___4_3m,cbdmedical___5_3m,cbdmedical___6_3m,cbdmedical___7_3m,cbdmedical___8_3m,cbdmedical___9_3m,cbdmedical___10_3m,cbdmedical___11_3m,cbdmedical___12_3m,cbdmedical___13_3m,cbdmedical___14_3m,cbdmedical___99_3m,oth_sym_3m,overallsym_scorebe_3m,overallsym_scoreaf_3m,moduse_3m,cbd_oth_3m,know_strain_3m,strain_3m,cbd_thc_3m,cbd_ratio_3m,recommcbd_3m,recom_oth_3m,obtain_3m,obtain_oth,cbdchange,howchng,stop_use___0,stop_use___1,stop_use___2,stop_use___3,stop_use___4,stop_useex,start_use___0,start_use___1,start_use___2,start_use___3,start_use___4,start_useex,cbd_change,whatchange___1,whatchange___2,whatchange___3,whatchange___4,whenchange,changefrequency,percent_crease,change_decrease,sympt_change,anynew,newsympt___1,newsympt___2,newsympt___3,newsympt___4,newsympt___5,newsympt___6,newsympt___7,newsympt___8,newsympt___9,newsympt___10,newsympt___11,newsympt___12,newsympt___13,newsympt___14,newsympt___0,newsympt_oth,symp_stop_bin,symp_stp___1,symp_stp___2,symp_stp___3,symp_stp___4,symp_stp___5,symp_stp___6,symp_stp___7,symp_stp___8,symp_stp___9,symp_stp___10,symp_stp___11,symp_stp___12,symp_stp___13,symp_stp___14,symp_stp___0,cbd_comment,eq5d_mb_5l_usa_eng_3m,eq5d_sc_5l_usa_eng_3m,eq5d_ua_5l_usa_eng_3m,eq5d_pd_5l_usa_eng_3m,eq5d_ad_5l_usa_eng_3m,eq5d5l_vas2_usa_eng_3m,in_pt_3m,dob_qol_3m,tod_3m,age_3m,c30_1_3m,c30_2_3m,c30_3_3m,c30_4_3m,c30_5_3m,c30_6_3m,c30_7_3m,c30_8_3m,c30_9_3m,c30_10_3m,c30_11_3m,c30_12_3m,c30_13_3m,c30_14_3m,c30_15_3m,c30_16_3m,c30_17_3m,c30_18_3m,c30_19_3m,c30_20_3m,c30_21_3m,c30_22_3m,c30_23_3m,c30_24_3m,c30_25_3m,c30_26_3m,c30_27_3m,c30_28_3m,c30_29_3m,c30_30_3m,bn20_31_3m,bn20_32_3m,bn20_33_3m,bn20_34_3m,bn20_35_3m,bn20_36_3m,bn20_37_3m,bn20_38_3m,bn20_39_3m,bn20_40_3m,bn20_41_3m,bn20_42_3m,bn20_43_3m,bn20_44_3m,bn20_45_3m,bn20_46_3m,bn20_47_3m,bn20_48_3m,bn20_49_3m,bn20_50_3m,sex_3m,visit_3m
record_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201,baseline_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Na

## To process in STATA we need a Tall dataset with the visit number

In [179]:
tall_df = pd.concat([filtered_df,fu1_df])
tall_df.sort_index()

,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,cbduse,currentuse,last_cdb,lenght,freqcbd,reascbd,other_use,cbdmedical___1,cbdmedical___2,cbdmedical___3,cbdmedical___4,cbdmedical___5,cbdmedical___6,cbdmedical___7,cbdmedical___8,cbdmedical___9,cbdmedical___10,cbdmedical___11,cbdmedical___12,cbdmedical___13,cbdmedical___14,cbdmedical___99,oth_sym,overallsym_scorebe,overallsym_scoreaf,moduse,cbd_oth,know_strain,strain,cbd_thc,cbd_ratio,recommcbd,recom_oth,obtain,eq5d_mb_5l_usa_eng,eq5d_sc_5l_usa_eng,eq5d_ua_5l_usa_eng,eq5d_pd_5l_usa_eng,eq5d_ad_5l_usa_eng,eq5d5l_vas2_usa_eng,in_pt,dob_qol,tod,age,c30_1,c30_2,c30_3,c30_4,c30_5,c30_6,c30_7,c30_8,c30_9,c30_10,c30_11,c30_12,c30_13,c30_14,c30_15,c30_16,c30_17,c30_18,c30_19,c30_20,c30_21,c30_22,c30_23,c30_24,c30_25,c30_26,c30_27,c30_28,c30_29,c30_30,bn20_31,bn20_32,bn20_33,bn20_34,bn20_35,bn20_36,bn20_37,bn20_38,bn20_39,bn20_40,bn20_41,bn20_42,bn20_43,bn20_44,bn20_45,bn20_46,bn20_47,bn20_48,bn20_49,bn20_50,ethnicity___1,ethnicity___2,ethnicity___3,ethnicity___4,ethnicity___5,ethnicity___6,ethnicity___7,education,employment,economic,ethn,race,maritalsts,vitalsts,last_alive,dod,sex,inital_dx,ini_surgery,ini_surgery_type,ini_tum_lateral,ini_tum_location,ini_path_dx,ini_tumor_grade,visit,obtain_oth,cbdchange,howchng,stop_use___0,stop_use___1,stop_use___2,stop_use___3,stop_use___4,stop_useex,start_use___0,start_use___1,start_use___2,start_use___3,start_use___4,start_useex,cbd_change,whatchange___1,whatchange___2,whatchange___3,whatchange___4,whenchange,changefrequency,percent_crease,change_decrease,sympt_change,anynew,newsympt___1,newsympt___2,newsympt___3,newsympt___4,newsympt___5,newsympt___6,newsympt___7,newsympt___8,newsympt___9,newsympt___10,newsympt___11,newsympt___12,newsympt___13,newsympt___14,newsympt___0,newsympt_oth,symp_stop_bin,symp_stp___1,symp_stp___2,symp_stp___3,symp_stp___4,symp_stp___5,symp_stp___6,symp_stp___7,symp_stp___8,symp_stp___9,symp_stp___10,symp_stp___11,symp_stp___12,symp_stp___13,symp_stp___14,symp_stp___0,cbd_comment
record_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201,baseline_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,1.0,2.0,95.0,AJ,1953-03-11,2018-08-22,66.062844,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,2.0,1.0,3.0,1.0,3.0,2.0,7.0,6.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,3.0,1.0,2.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,8.0,2.0,1.0,1.0,0.0,0.0,NaN,NaN,0.0,08/09/1995,1.0,1.0,2.0,2.0,Oligodendroglioma,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,3_mo_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,2.0,2.0,89.0,Alwin,1953-03-11,2018-12-01,NaN,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,3.0,6.0,6.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
202,3_mo_arm_1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,85.0,SA,1957-09-06,2018-12-04,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,6.0,7.0

In [180]:
tall_df.to_csv('SNO 2021.csv')

Checks how many subjects have completed the 6 month follow up

In [94]:
fu2_event=['6_mo_arm_1']
fu2_df = project.export_records(format='df', raw_or_label='raw', export_checkbox_labels=True,events=fu2_event, fields=fields_oi, filter_logic="[eortcquality_of_life_complete]=2")
len(fu2_df.index)

45

In [183]:
tall_df[eq5d_ua].head()

NameError: name 'eq5d_ua' is not defined